In [2]:
import spacy
import random
import torch
from transformers import pipeline
from transformers import RobertaTokenizer, RobertaForMaskedLM

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
model = RobertaForMaskedLM.from_pretrained('roberta-large').to(device)

nlp = spacy.load("en_core_web_sm")
pos_set = ['ADJ', 'ADV', 'INTJ', 'NOUN', 'PRON', 'VERB', 'ADP', 'AUX', 'CCONJ', 'DET', 'PRON', 'SCONJ', 'X']

Using device: cuda


In [3]:
def synonym_replacement(text, p1):
    doc = nlp(text)
    result_words = []
    
    for token in doc:
        if not token.is_stop and token.pos_ in pos_set and random.random() < p1:
            masked_text = text[:token.idx] + '<mask>' + text[token.idx + len(token.text):]
            # print(masked_text)
            input_ids = tokenizer.encode(masked_text, return_tensors='pt').to(device)
            
            with torch.no_grad():
                outputs = model(input_ids)
            
            mask_index = torch.where(input_ids[0] == tokenizer.mask_token_id)[0]
            predictions = outputs[0][0, mask_index, :].topk(3)
            predicted_token_indexes = predictions[1].tolist()

            predicted_token_index = random.choice(predicted_token_indexes[0])
            predicted_token = tokenizer.decode([predicted_token_index]).strip()
            # print(f"{token.text} --> {predicted_token}")
            
            result_words.append(predicted_token)
        else:
            result_words.append(token.text)
    
    sentence = ''
    for word in result_words:
        if sentence and word in ',.;!?':
            sentence += word
        else:
            sentence += (' ' + word if sentence else word)
    
    return sentence

In [4]:
def random_insertion(text, p2):
    doc = nlp(text)
    result_words = text.split()
    insertions = []
    
    for token in doc:
        if not token.is_stop and token.pos_ in pos_set and random.random() < p2:
            masked_text = text[:token.idx] + '<mask>' + text[token.idx + len(token.text):]
            # print(masked_text)
            input_ids = tokenizer.encode(masked_text, return_tensors='pt').to(device)
            
            with torch.no_grad():
                outputs = model(input_ids)
            
            mask_index = torch.where(input_ids[0] == tokenizer.mask_token_id)[0]
            predictions = outputs[0][0, mask_index, :].topk(3)
            predicted_token_indexes = predictions[1].tolist()

            predicted_token_index = random.choice(predicted_token_indexes[0])
            predicted_token = tokenizer.decode([predicted_token_index]).strip()
            # print(f"Inserting: {token.text} --> {predicted_token}")
            
            insertions.append(predicted_token)
    
    for insertion in insertions:
        insert_position = random.randint(0, len(result_words))
        result_words.insert(insert_position, insertion)
    
    return ' '.join(result_words)

In [5]:
def random_swap(text, p3):
    doc = nlp(text)
    
    valid_tokens = [token for token in doc if not token.is_stop and token.pos_ in pos_set]
    swap_cnt = int(len(valid_tokens) * p3)
    
    for _ in range(swap_cnt):
        token1 = random.choice(valid_tokens)
        token2 = random.choice(valid_tokens)
        
        text = text.replace(token1.text, "_TEMP_", 1)
        text = text.replace(token2.text, token1.text, 1)
        text = text.replace("_TEMP_", token2.text, 1)
    
    return text

In [6]:
def random_deletion(text, p4):
    doc = nlp(text)
    result_words = [token.text for token in doc if token.is_stop or random.random() >= p4]
    
    sentence = ''
    for word in result_words:
        if sentence and word in ',.;!?':
            sentence += word
        else:
            sentence += (' ' + word if sentence else word)
    
    return sentence

In [28]:
test_1 = "This game sets a new benchmark for open-world exploration, with a sprawling gallery of monsters each with their own unique moveset, secrets, puzzles and challenging bosses all with intricately woven lore and questlines. The map is vast end littered with content that will keep you engaged throughout the game. From Software has truely made a compelling and challenging game that leaves you wanting more."

In [29]:
print(synonym_replacement(test_1, 0.5))
print(random_insertion(test_1, 0.5))
print(random_swap(test_1, 0.5))
print(random_deletion(test_1, 0.5))

This RPG sets a new benchmark for open - world exploration, with a vast cast of monsters each with their own unique moveset, secrets, puzzles and challenging bosses all with deeply woven lore and questlines. The map is vast end littered with surprises that will keep you busy throughout the game. From Software has truely made a unique and ambitious game that leaves you wanting more.
This bar game sets a new benchmark for open-world exploration, with a sprawling experience gallery of diverse monsters quirks each with bosses their own encounters unique games moveset, secrets, packed puzzles and challenging bosses all with intricately woven lore and questlines. The map is vast end boss littered stories with content that will keep variety you traps engaged throughout the game. From Software has truely made a again compelling and challenging game RPG that leaves you wanting more.
This puzzles sets a open engaged for moveset-world leaves, with a game gallery of woven each with their own uniqu

In [326]:
import json

with open('train_1.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

enhanced_data = {}
count = 0

for review_id, review_data in data.items():
    if count % 100 ==0:
        print(count)
    count += 1
    
    enhanced_data[review_id] = review_data
    text = review_data['text']

    if review_data['audio'] == 1:
        enhanced_data[review_id + '_sr'] = {
            **review_data,
            'text': synonym_replacement(text, 0.1)
        }
        enhanced_data[review_id + '_ri'] = {
            **review_data,
            'text': random_insertion(text, 0.1)
        }
        enhanced_data[review_id + '_rw'] = {
            **review_data,
            'text': random_swap(text, 0.1)
        }
        enhanced_data[review_id + '_rd'] = {
            **review_data,
            'text': random_deletion(text, 0.1)
        }
    elif review_data['items'] == 1 or review_data['narrative'] == 1:
        enhanced_data[review_id + '_sr'] = {
            **review_data,
            'text': synonym_replacement(text, 0.1)
        }
        enhanced_data[review_id + '_ri'] = {
            **review_data,
            'text': random_insertion(text, 0.1)
        }
    

with open('train_3.json', 'w', encoding='utf-8') as file:
    json.dump(enhanced_data, file, indent=4)
print("finished!")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
finished!
